In [ ]:
# vtreat

# Setting up vtreat for a binomial classification problem
treatment = vtreat.BinomialOutcomeTreatment(
    outcome_name='readmitted',
    outcome_target=1,  # Target for binary classification
    cols_to_copy=None,  # Copy without any transformation
    params={
        'filter_to_recommended': True,
        'indicator_min_fraction': 0.01
    }
)

# Fitting the treatment to the training data
Xy_train = pd.concat([X_train, y_train], axis=1)
train_treated = treatment.fit_transform(Xy_train)

# Transform the test data using the same treatment
Xy_test = pd.concat([X_test, y_test], axis=1)
test_treated = treatment.transform(Xy_test)

# examine score frame
# treatment.score_frame_

# recommended variables
# treatment.score_frame_.loc[treatment.score_frame_['recommended'], ['variable']]

# not recommended variables
# treatment.score_frame_.loc[~treatment.score_frame_['recommended'], ['variable']]

In [ ]:
#data = (train_treated.drop(columns=['readmitted']), 
#        test_treated.drop(columns=['readmitted']), 
#        train_treated['readmitted'], 
#        test_treated['readmitted'])

In [ ]:
#gb.fit(train_treated.drop(columns=['readmitted']), train_treated['readmitted']) # 0.721

In [ ]:
####################################
########## SIMULATED DATA ##########
####################################

from sklearn import datasets

random_state = 109

# create dataset for binary classification with 5 predictors
X, y = datasets.make_classification(
    n_samples=1000,
    n_features=5,
    n_informative=3,
    n_redundant=2,
    random_state=random_state
)

# split into training and test
X_train_transformed, X_test_transformed, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8,
    random_state=random_state
)

X_train_transformed = pd.DataFrame(X_train_transformed, columns=['one', 'two', 'three', 'four', 'five'])
X_test_transformed = pd.DataFrame(X_test_transformed, columns=['one', 'two', 'three', 'four', 'five'])

In [ ]:
# Lasso Logistic Regression

# C values to try 
Cs = [1e-1, 1e0, 1e1, 1e2, 1e3]

# logistic regression model with elasticnet penalty and 10-fold CV
lrr = LogisticRegressionCV(
    Cs=Cs, cv=10, penalty='elasticnet', solver='saga', n_jobs=-1, # solver='liblinear' for l1
    scoring='roc_auc', max_iter=1000, random_state=109, l1_ratios=[0, 0.5, 1],
)

start_time = time.time()

lrr.fit(X_train_transformed, y_train)

end_time = time.time()
print(f"Model fitting time: {end_time - start_time:.1f} seconds")

In [ ]:
best_C = lrr.C_
print("Best C for each class: ", best_C)

In [ ]:
# get scores
scores = lrr.scores_

# Calculate mean scores across all folds
mean_scores = {class_label: score.mean(axis=0) for class_label, score in scores.items()}

# Print mean scores
print("Mean Validation Scores: ", mean_scores)